In [11]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)


def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, width, height, channels]
    # MNIST images are 28x28 pixels, and have one color channel
    input_layer = tf.reshape(features["x"], [-1, 128, 28, 1])

    # Convolutional Layer #1
    # Computes 32 features using a 5x5 filter with ReLU activation.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 28, 28, 1]
    # Output Tensor Shape: [batch_size, 28, 28, 32]
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 28, 28, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2
    # Computes 64 features using a 5x5 filter.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 14, 14, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 64]
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #2
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 14, 14, 64]
    # Output Tensor Shape: [batch_size, 7, 7, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [batch_size, 7, 7, 64]
    # Output Tensor Shape: [batch_size, 7 * 7 * 64]
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

    # Dense Layer
    # Densely connected layer with 1024 neurons
    # Input Tensor Shape: [batch_size, 7 * 7 * 64]
    # Output Tensor Shape: [batch_size, 1024]
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

    # Add dropout operation; 0.6 probability that element will be kept
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 10]
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


def main(unused_argv):
    # Load training and eval data
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    train_data = mnist.train.images  # Returns np.array
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    eval_data = mnist.test.images  # Returns np.array
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

    # Create the Estimator
    mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

    # Set up logging for predictions
    # Log the values in the "Softmax" tensor with label "probabilities"
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
    mnist_classifier.train(
      input_fn=train_input_fn,
      steps=20000,
      hooks=[logging_hook])

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

if __name__ == "__main__":
    tf.app.run()

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x121971eb8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.10119652  0.10873593  0.09963275  0.09874225  0.09800243  

INFO:tensorflow:loss = 2.31609, step = 1
INFO:tensorflow:probabilities = [[ 0.09496771  0.10551788  0.09499744  0.09604934  0.09464189  0.09713991
   0.10482781  0.10495456  0.10274883  0.10415469]
 [ 0.10636753  0.09785914  0.10701583  0.09131967  0.0940451   0.10920069
   0.09798934  0.10113507  0.10347304  0.09159456]
 [ 0.11932434  0.08850795  0.09912644  0.09203109  0.09446617  0.10090774
   0.11034068  0.09360567  0.09886836  0.10282157]
 [ 0.10822158  0.08463757  0.10852851  0.08941022  0.09551288  0.10911919
   0.10841752  0.11048904  0.09613807  0.08952551]
 [ 0.11036343  0.1022815   0.0902211   0.09324874  0.09340739  0.09742326
   0.09380796  0.1016032   0.11256807  0.10507534]
 [ 0.10271245  0.09934565  0.09776451  0.09534497  0.10022467  0.09954128
   0.09894971  0.10107738  0.097534    0.10750537]
 [ 0.11092908  0.09201956  0.09908839  0.09963336  0.09251525  0.10375602
   0.10721065  0.0988534   0.09886375  0.09713051]
 [ 0.10244767  0.10719561  0.08920992  0.09529585  0

INFO:tensorflow:global_step/sec: 2.74999
INFO:tensorflow:probabilities = [[ 0.10735097  0.09533235  0.09976765  0.09969304  0.09386881  0.10142221
   0.10017084  0.09437414  0.10185797  0.10616208]
 [ 0.10823157  0.09054787  0.10368498  0.10323229  0.10022669  0.10883357
   0.09295722  0.09795441  0.09743345  0.09689789]
 [ 0.10873149  0.10281241  0.09479167  0.09491349  0.08841612  0.10573997
   0.10491613  0.09996168  0.09966063  0.10005637]
 [ 0.10322154  0.10120772  0.09412602  0.09948219  0.10934225  0.09675926
   0.09496417  0.09985071  0.10421745  0.09682856]
 [ 0.11003383  0.10474432  0.1085261   0.09781957  0.08622473  0.1038465
   0.09870643  0.09005193  0.10719463  0.092852  ]
 [ 0.106284    0.10679369  0.09633693  0.10606456  0.08644408  0.10672703
   0.10006319  0.1003404   0.10015637  0.09078977]
 [ 0.09826908  0.09558411  0.0956267   0.11012454  0.09839504  0.10020508
   0.09609411  0.09826744  0.11003856  0.09739534]
 [ 0.11001289  0.09894972  0.09526482  0.09685829  0.

INFO:tensorflow:loss = 2.29908, step = 101 (36.365 sec)
INFO:tensorflow:probabilities = [[ 0.10378005  0.09170397  0.11249231  0.08306099  0.09179293  0.1068555
   0.10322651  0.10064997  0.10394394  0.10249381]
 [ 0.1115613   0.10299467  0.09219026  0.09513423  0.09550052  0.10332758
   0.09792981  0.10138438  0.09911951  0.10085776]
 [ 0.1000192   0.09857919  0.09805644  0.10525975  0.09579352  0.10045713
   0.09840276  0.10216736  0.09904744  0.1022172 ]
 [ 0.09906471  0.09431395  0.09255715  0.10285024  0.10825022  0.10637647
   0.0914254   0.10632728  0.10530252  0.09353203]
 [ 0.09215972  0.10220358  0.11172979  0.09981942  0.10014782  0.1004104
   0.09982836  0.10462033  0.1035866   0.08549402]
 [ 0.09459554  0.09157187  0.10737744  0.10226867  0.09493515  0.10338718
   0.10179905  0.1002315   0.10180534  0.10202821]
 [ 0.11209546  0.08893888  0.11102179  0.09772573  0.08377678  0.10032044
   0.1095339   0.09439757  0.1009946   0.10119487]
 [ 0.10728913  0.10089346  0.09248012  

INFO:tensorflow:global_step/sec: 2.7902
INFO:tensorflow:probabilities = [[ 0.10018022  0.10226602  0.09060315  0.09592437  0.09642281  0.10021652
   0.11409381  0.09611925  0.10724393  0.09692996]
 [ 0.10909712  0.09542955  0.10511538  0.09055287  0.0906411   0.100904
   0.10594088  0.10338895  0.09397562  0.10495459]
 [ 0.09353853  0.10988117  0.0996525   0.08956128  0.09505031  0.09927814
   0.09780557  0.10993302  0.10581195  0.09948749]
 [ 0.10088168  0.09171896  0.09496715  0.10267548  0.09943204  0.10028818
   0.10962309  0.10064614  0.09850559  0.10126177]
 [ 0.10629699  0.09404378  0.10849509  0.09235547  0.09175079  0.09733788
   0.09732612  0.09375259  0.10773449  0.11090678]
 [ 0.1092691   0.10204195  0.10102868  0.09154666  0.09450286  0.10881032
   0.10662305  0.09787216  0.1024154   0.08588983]
 [ 0.10945553  0.10088772  0.09825918  0.09939624  0.0967019   0.10112688
   0.09711298  0.09582221  0.10342022  0.09781718]
 [ 0.10682119  0.08924584  0.09063146  0.10629556  0.08

INFO:tensorflow:loss = 2.27751, step = 201 (35.837 sec)


KeyboardInterrupt: 